# Projeto 1 - Ciência dos Dados

Nome: Letícia Coêlho Barbosa

Nome: Matheus Silva Melo de Oliveira

## Contextualização

------------------------------------------------------------

## Bibliotecas Utilizadas

------------------------------------------------------------

In [376]:
#!pip install emoji
#!pip install pysinonimos

Carregando algumas bibliotecas:

In [505]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

import re 
import emoji
import pysinonimos.sinonimos as sn
import nltk
from emoji import UNICODE_EMOJI

Confirmando diretório de trabalho

In [506]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\mathe_vz6v46f\OneDrive\Documentos\Arquivos INSPER 2° SEMESTRE\C-Dados\TheMandalorian


## Banco de Dados

------------------------------------------------------------

Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [507]:
filename = 'mandalorian.xlsx'

In [508]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevancia
0,executivo da disney defendeu demissão de gina ...,0
1,@daredvevil nao seriooo melhor dia 😭 volta the...,1
2,preciso terminar mandalorian logo,1
3,toda vez que eu vejo o baby yoda usando a forç...,0
4,jon fraveau eu te amo obrigado por fazer the m...,1


In [509]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevancia
0,the mandalorian é uma série com dinheiro e tec...,1
1,agora comecei mandalorian confiando novamente ...,1
2,@fepazo1 @gabriel_ns1999 @joelmsimmons @jurand...,1
3,eu assisti 3 minutos de mandalorian e emocionei,1
4,rt @anakinwar: já ta ...,0


## Classificador automático de sentimento

------------------------------------------------------------

Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.


___
### Definindo Funções


In [510]:
#Limpando os tweets retirando pontos desnecessarios
def limpeza(tweet):
    punctuation = '[!-.:?;\n|]'                # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern,'', tweet)
    return text_subbed

#Limpando os tweets retirando emojis
def retira_emoji(content: str):
        modified = re.sub(emoji.get_emoji_regexp(), r"", content)
        modified = re.sub(':[^>]+:', '', modified)
        modified = re.sub('<[^>]+>', '', modified)
        return modified.strip() 

#Armazenando listas com preposições na língua portuguesa
nltk.download('stopwords')
prep = nltk.corpus.stopwords.words('portuguese')
prep.append('')

# Define função que aplica remoção de preposições
def limpa_preposicao(lista):
    sem_prep=[]     
    for palavra in lista: 
        if palavra in prep: 
            continue
        else:
            sem_prep.append(palavra)
            
    return sem_prep
    
# Transformando tweets em lower case:
def lower(tweet):
    return tweet.lower()

#retira escritas comuns em tweets, mas irrelevantes para análise de dados
def tweet_to_frase(lista):
    for i,word in enumerate(lista):
        if 'rt' in word or 'https' in word or '@' in word:
            lista[i]=lista[i].replace(word,'null')
    new_list=[x for x in lista if x!='null']
    return new_list

def transcreve_emoji(lista):
    emojis=re.findall(emoji.get_emoji_regexp(),lista)                   #Tornando os emojis em strings descritivas
    for i,emoji1 in enumerate(emojis):                              #Alguns emojis não possuem transcrição para português
            if emoji1 in UNICODE_EMOJI['pt']:
                emojis[i]=UNICODE_EMOJI['pt'][emoji1].replace(':',',')
            else:
                emojis[i]=UNICODE_EMOJI['en'][emoji1].replace(':',',')
    return emojis

#Transformando lista em string
def list_to_string(lista):
    return ','.join(lista)

#Transformando string em lista
def string_to_list(string):
    return string.split()

   
# Concatenando tweets palavras+emojis limpos:
def aplica(tweet):
    
    #captando emojis
    emoji_transcrito=list_to_string(transcreve_emoji(tweet))
    
    #aplicando todas as funções definidas
    frase_separada=list_to_string(tweet_to_frase(limpa_preposicao(string_to_list(limpeza(lower(retira_emoji(tweet)))))))
    
    #retornando uma string limpa e com emojis transcritos
    return frase_separada+','+emoji_transcrito

def frases_to_words (Serie_frases):
    
    texto=''
    
    for palavra in Serie_frases:
        texto+=str(palavra)                        #Criando uma string gigante com todas as palavras das frases
    
    lista_palavras=texto.split(',')                #Criando lista das palavras contidas na variável texto
    return pd.Series(lista_palavras)               #Retornando uma série de palavras contida nas frases

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mathe_vz6v46f\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


___
### Montando um Classificador Naive-Bayes



Considerando apenas as mensagens da planilha Treinamento:

In [511]:
#Transformando palavras em variáveis categóricas:
train['Treinamento'] = train['Treinamento'].astype('category')
test['Teste'] = test['Teste'].astype('category')

In [512]:
#Aplicando funções de limpeza:
train['Clean']=train['Treinamento'].apply(aplica)

test['Clean']=test['Teste'].apply(aplica)


In [513]:
#Separando dados de treino por relevância:

train_relevantes=train[train['Relevancia']==1]
train_irrelevantes=train[train['Relevancia']==0]

In [514]:
#Serie de palavras relevantes e irrelevantes no conjunto treino:

palavras_r= frases_to_words(train_relevantes['Clean'])
palavras_i= frases_to_words(train_irrelevantes['Clean'])

In [515]:
# Conferindo resultado:

palavras_r

0                                 nao
1                             seriooo
2                              melhor
3                                 dia
4                               volta
                    ...              
3122                              the
3123                      mandalorian
3124                                 
3125    mãos_juntas_pele_morena_clara
3126                                 
Length: 3127, dtype: object

In [516]:
#Criando texto com palavras relevantes:

texto_r=''

for palavra in train_relevantes['Clean']:
    texto_r+=str(palavra)
#print(texto_r)

In [517]:
#Criando texto com palavras irrelevantes:
texto_i=''

for palavra in train_irrelevantes['Clean']:
    texto_i+=str(palavra)


In [518]:
# Frequência relativa das palavras:

rel_relevantes=palavras_r.value_counts(normalize=True)
rel_irrelevantes=palavras_i.value_counts(normalize=True)

In [519]:
# Conferindo resultado:

In [520]:
#Para usar em laplace:
list_i=rel_irrelevantes.index.tolist()
list_r=rel_relevantes.index.tolist()
elementos_nao_repetidos=set(list_i+list_r)

In [521]:
#Série representante do conjunto universo de todas as palavras:
palavras_i=palavras_i.tolist()
palavras_r=palavras_r.tolist()
#total=palavras_i+palavras_r
total=pd.Series(palavras_i+palavras_r)             
total

0                           executivo
1                              disney
2                            defendeu
3                            demissão
4                                gina
                    ...              
5598                              the
5599                      mandalorian
5600                                 
5601    mãos_juntas_pele_morena_clara
5602                                 
Length: 5603, dtype: object

In [522]:
#Frequência das palvras no total:
rel_total=total.value_counts(True)

In [523]:
#Conferindo resultado
rel_total

mandalorian    0.080493
               0.046225
the            0.041406
pra            0.010530
wars           0.008924
                 ...   
diversidade    0.000178
decidi         0.000178
pelúcia        0.000178
batalha        0.000178
perdido        0.000178
Length: 2002, dtype: float64

### Montando as Probabilidades 

$$ P_{relevante}= \frac{Palavras_{relevantes}}{Total_{palavras}} $$

$$ P_{irrelevante}= \frac{Palavras_{irrelevantes}}{Total_{palavras}} $$

Onde, sabemos que:

$$ P_{relevante}+ P_{irrelevante}=1 $$

In [524]:
#Probabilidade tweet ser relevante ou irrelevante:
P_r=len(palavras_r)/len(total)
P_i=len(palavras_i)/len(total)

Ingenuidade do classificador Navie Bayes

$$ P(R|tweet)= \frac{P(tweet \cap R)}{P(tweet)} $$
$$ P(tweet|R)= \frac{P(tweet \cap R)}{P(R)} $$

$$ P(tweet \cap R) = P(R|tweet) P(tweet) $$
$$ P(tweet \cap R) = P(tweet|R) P(R) $$

Sendo a Probabiliadde de Navie Bayes:
    

$$ P(R|tweet)= \frac{P(tweet|R) P(R)}{P(tweet)} $$

A probabilidade $P(tweet|R)$, pode ser calculada considerando a independência entre as palavras, ou seja, a probabilidade de certa palavra aparecer não interfere na probabilidade de outras:

$$ P(tweet|R) = P(palavra1|R)P(palavra2|R)...P(palavraN|R)$$

Temos portayndo como porbabilidade de um tweet ser relevante:

$$ P(R|tweet) = \frac{P(palavra1|R)P(palavra2|R)...P(palavraN|R) P(R)}{P(tweet)}$$

Analogamento, para $P(I|tweet)$, teriamos:

$$P(I|tweet)=\frac{P(palavra1|I)P(palavra2|I)...P(palavraN|I) P(I)}{P(tweet)}$$

Portanto, a função Navie Bayes classificaria como relevante caso:

$$P(R|tweet)>P(I|tweet)$$

In [568]:
lista_r=['mandalorian','de','the','é','que','a','e','o','eu','da','mas','com']
lista_i=['mandalorian','de',' ','the','e','o','a','que','do','é','eu','da','em','no','um','pra','com']
lista_r_new=[x for x in lista_r if x not in prep]
lista_i_new=[x for x in lista_i if x not in prep]

In [569]:
def teste(palavra,r):
    count=0
    for i in range(0,len(r)):
        if palavra==r[i]:
            count+=1
        else:
            continue
    return count+1
teste('augusto',['ele'])            
    
list_i=rel_irrelevantes.index.tolist()
list_r=rel_relevantes.index.tolist()
elementos_nao_repetidos=set(list_i+list_r)
teste('augusto',['ele']) 

1

In [581]:
def P_tweet_r (tweet):
    
    c=1
    r=tweet.split(',')
    #r=[x for x in r if x!='mandalorian']
    for palavra in r:
        
        x=teste(palavra,list_r)
        k=x/(len(list_r)+len(elementos_nao_repetidos))
        c*=k
#         if(palavra in palavras_r) and (palavra not in lista_r_new) :
            
#             c*=rel_relevantes[palavra]
#         elif palavra in rel_total:
#             c*=rel_total[palavra]
#             #c*=dicio_r(palavra)
#         else:
#             continue
           
    return c

In [582]:
def P_tweet_i (tweet):
    
    
    c=1
    
    r=tweet.split(',')
    #r=[x for x in r if x!='mandalorian']
    for palavra in r:
        
        x=teste(palavra,list_i)
        k=x/(len(list_i)+len(elementos_nao_repetidos))
        c*=k
    
#     for palavra in tweet.split(','):
#         if(palavra in palavras_i) and (palavra not in lista_i_new):
#             c*=rel_irrelevantes[palavra]
        
#         elif palavra in rel_total: 
#             c*=rel_total[palavra]
#             #c*=dicio_i(palavra)
#         else:
#             continue
        
    return c

In [583]:
#Criando Nossa fução Naive_Bayes:

def Naive_Bayes (tweet):
    if(P_tweet_r(tweet)*P_r > P_tweet_i(tweet)*P_i):
        return 1
    else:
        return 0

In [584]:
 P_tweet_r('dps,de,insistirem,mt,estou,começando,mandalorian,espero,q,realmente,seja,o,caso,de,salvação,do,star,wars,alegria,Michel')

7.010494842759616e-67

In [585]:
P_tweet_i('dps,de,insistirem,mt,estou,começando,mandalorian,espero,q,realmente,seja,o,caso,de,salvação,do,star,wars,alegria, Michel')

1.7828817168946249e-68

In [586]:
#Criando uma nova coluna para armazenar resultado da nossa função Navie_Bayes

train['Classificador']=train.Clean.apply(Naive_Bayes)

In [587]:
train 

,Treinamento,Relevancia,Clean,Classificador
0,executivo da disney defendeu demissão de gina ...,0,"executivo,disney,defendeu,demissão,gina,carano...",0
1,@daredvevil nao seriooo melhor dia 😭 volta the...,1,"nao,seriooo,melhor,dia,volta,the,mandalorian,p...",1
2,preciso terminar mandalorian logo,1,"preciso,terminar,mandalorian,logo,",1
3,toda vez que eu vejo o baby yoda usando a forç...,0,"toda,vez,vejo,baby,yoda,usando,força,the,manda...",0
4,jon fraveau eu te amo obrigado por fazer the m...,1,"jon,fraveau,amo,obrigado,fazer,the,mandalorian...",1
...,...,...,...,...
495,"último ep de mandalorian, não estou preparada",1,"último,ep,mandalorian,preparada,",1
496,minha prioridade agora é terminar the mandalorian,1,"prioridade,agora,terminar,the,mandalorian,",1
497,agora vou começar a ver the mandalorian 🙏🏼,1,"agora,vou,começar,ver,the,mandalorian,,mãos_ju...",1
498,"@jurandirfilho tem nem comparação né man, mand...",0,"comparação,né,man,mandalorian,",0


In [588]:
#Porgentagem de acerto no conjunto de Treino:

x=train.loc[(train['Classificador']==1)&(train['Relevancia']==1),:].shape[0]
y=train.loc[(train['Classificador']==0)&(train['Relevancia']==0),:].shape[0]
(x+y)/train.shape[0]

0.954

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [589]:
test['Classificador']=test.Clean.apply(Naive_Bayes)

In [590]:
x=test.loc[(test['Classificador']==1)&(test['Relevancia']==1),:].shape[0]
y=test.loc[(test['Classificador']==0)&(test['Relevancia']==0),:].shape[0]
(x+y)/test.shape[0]

0.5244444444444445

In [591]:
# fp=test.loc[(test['Classificador']==0)&(test['Relevancia']==1),:].shape[0]
# tp=test.loc[(test['Classificador']==1)&(test['Relevancia']==1),:].shape[0]
# fn=test.loc[(test['Classificador']==1)&(test['Relevancia']==0),:].shape[0]
# tn=test.loc[(test['Classificador']==0)&(test['Relevancia']==0),:].shape[0]

# test['Classificador']=
pd.crosstab(test.Relevancia,test.Classificador,normalize=True)

Classificador,0,1
Relevancia,,
0,0.168889,0.377778
1,0.097778,0.355556


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


In [60]:
def dicio_r (palavra):
    
    nova=sn.Search(palavra).synonyms()   #Buscando sinônimos
    P_nova=0
    
    if nova != 404:
        
        for i in range(len(nova)):
        
            if (i != len(nova)-1):
                if (nova[i] in palavras_r) and (nova[i] not in lista_r):
                    P_nova=rel_relevantes[nova[i]]
                    return P_nova
                else:
                    continue

            else:
                return 1
    else:
        return 1
        

In [ ]:
def dicio_i (palavra):
    
    nova=sn.Search(palavra).synonyms()   #Buscando sinônimos
    P_nova=0
    
    if nova != 404:
    
        for i in range(len(nova)):
            
            if (i != len(nova)-1):
            
                if (nova[i] in palavras_r) and (nova[i] not in lista_i):
                    P_nova=rel_relevantes[nova[i]]
                    return P_nova
                else:
                    continue
            
            else:
                return 1
    else:
        return 1
        

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**